In [54]:
import pandas as pd
import numpy as np
from datetime import datetime, time


Import the .csv file

In [55]:
df = pd.read_csv("GBPUSD15_dirty.csv")
df.head(10), df.dtypes

(   1993.05.12  00:00  1.53700  1.54450  1.52900  1.53380  2781
 0  1993.05.13  00:00   1.5328   1.5360   1.5180   1.5225  2571
 1  1993.05.14  00:00   1.5228   1.5415   1.5200   1.5387  2711
 2  1993.05.17  00:00   1.5365   1.5460   1.5309   1.5355  2921
 3  1993.05.18  00:00   1.5350   1.5380   1.5237   1.5365  2711
 4  1993.05.19  00:00   1.5350   1.5482   1.5328   1.5432  2261
 5  1993.05.20  00:00   1.5425   1.5603   1.5383   1.5565  3001
 6  1993.05.21  00:00   1.5548   1.5592   1.5390   1.5425  2811
 7  1993.05.24  00:00   1.5400   1.5450   1.5289   1.5365  2871
 8  1993.05.25  00:00   1.5385   1.5470   1.5345   1.5420  2151
 9  1993.05.26  00:00   1.5420   1.5505   1.5410   1.5472  1381,
 1993.05.12     object
 00:00          object
 1.53700       float64
 1.54450       float64
 1.52900       float64
 1.53380       float64
 2781            int64
 dtype: object)

Format the dataframe to make it easier to work with


In [56]:
# column names
df.columns = ["date", "time", "open", "high", "low", "close", "volume"]
# add a datetime column
df["datetime"] = df["date"] + " " + df["time"]
# datetime format
df["date"] = pd.to_datetime(df["date"])
df["time"] = pd.to_datetime(df["time"], format="%H:%M").dt.time
df["datetime"] = pd.to_datetime(df["datetime"])
df["DoW"] = df["date"].dt.day_name()


df.set_index("datetime", inplace=True)
df["DoW"] = df.index.dayofweek

# reduce dataset to greater than 2015
start_date = "01-01-2022"
df = df[df["date"] > start_date]
df.head()

,date,time,open,high,low,close,volume,DoW
datetime,,,,,,,,
2022-01-03 00:00:00,2022-01-03,00:00:00,1.35256,1.35268,1.35238,1.35246,50,0
2022-01-03 00:15:00,2022-01-03,00:15:00,1.35243,1.35306,1.35215,1.35246,94,0
2022-01-03 00:30:00,2022-01-03,00:30:00,1.35267,1.35336,1.35237,1.35286,153,0
2022-01-03 00:45:00,2022-01-03,00:45:00,1.35284,1.35319,1.35265,1.35281,95,0
2022-01-03 01:00:00,2022-01-03,01:00:00,1.35302,1.35328,1.35250,1.35254,231,0


In [57]:
df.dtypes

date      datetime64[ns]
time              object
open             float64
high             float64
low              float64
close            float64
volume             int64
DoW                int64
dtype: object

Add the information about the timed range that is being analyzed

In [58]:
# asian session end and length
asia_start = datetime.strptime("03:00:00", "%H:%M:%S").time()
asia_end = datetime.strptime("07:00:00", "%H:%M:%S").time()

ldn_end = datetime.strptime("15:00:00", "%H:%M:%S").time()
asia_length = 16
print(asia_start, asia_end)

03:00:00 07:00:00


Main section of code where the asian range is found and checked if the high to low is taken (vice versa for bullish)

In [59]:
# Define useful global variables and empty lists
asia_high = 10
asia_low = 0
seek_hi_lo = []
seek_lo_hi = []
hi_taken = []
lo_taken = []
seek_low = 0
seek_high = 0

# Assuming you have a pandas DataFrame named df
for i in range(len(df)):
    # Slice the data to find the range high and low
    if df['time'].iloc[i] == asia_end:

        asia_range = df.iloc[i - asia_length:i]
        asia_high = asia_range['high'].max()
        asia_low = asia_range['low'].min()
    # time bracket to check the profile occurs
    elif df['time'].iloc[i] > asia_end and df['time'].iloc[i] < ldn_end:
        # bearish case
        if df['high'].iloc[i] > asia_high:
            # Store the date of when the high was taken
            hi_taken.append(df['date'].iloc[i])
            seek_low = 1
        # bullish case
        elif df['low'].iloc[i] < asia_low:
            lo_taken.append(df['date'].iloc[i])
            seek_high = 1
        # objective met
        if seek_low ==1 and df['low'].iloc[i] < asia_low:
            seek_hi_lo.append(df['date'].iloc[i] )
        elif seek_high ==1 and df['high'].iloc[i] > asia_high:
            seek_lo_hi.append(df['date'].iloc[i])
    elif df['date'].iloc[i] != df['date'].iloc[i - 1]:

        # reset variables at the end of each day
        asia_high = 10
        asia_low = 0
        seek_low = 0
        seek_high = 0

In [60]:
bear_examples = len(set(seek_hi_lo))
bullish_examples= len(set(seek_lo_hi))
total_examples = bear_examples + bullish_examples
dates = pd.to_datetime(df['date']).dt.date.unique().tolist()
sample_size = len(dates)
chances = (total_examples / sample_size) * 100
chances, sample_size, bear_examples, bullish_examples

(65.92592592592592, 270, 91, 87)

Now that we have the probabilities of it occuring lets print what a day looks like so we know what we searched for actually happend


In [61]:
example_dates = set(seek_lo_hi)
example_date = list(example_dates)[-1]
example_date


Timestamp('2022-09-05 00:00:00')

Find the OHCL values for a day that was found

In [65]:
date = pd.to_datetime(example_date)
day = df[df['date'] == date]
day

,date,time,open,high,low,close,volume,DoW
datetime,,,,,,,,
2022-09-05 00:00:00,2022-09-05,00:00:00,1.14797,1.14891,1.14797,1.14888,80,0
2022-09-05 00:15:00,2022-09-05,00:15:00,1.14888,1.14984,1.14833,1.14866,194,0
2022-09-05 00:30:00,2022-09-05,00:30:00,1.14873,1.14918,1.14798,1.14885,122,0
2022-09-05 00:45:00,2022-09-05,00:45:00,1.14888,1.14925,1.14846,1.14848,120,0
2022-09-05 01:00:00,2022-09-05,01:00:00,1.14859,1.14935,1.14782,1.14806,851,0
...,...,...,...,...,...,...,...,...
2022-09-05 22:45:00,2022-09-05,22:45:00,1.15159,1.15175,1.15136,1.15167,247,0
2022-09-05 23:00:00,2022-09-05,23:00:00,1.15167,1.15174,1.15134,1.15163,247,0
2022-09-05 23:15:00,2022-09-05,23:15:00,1.15163,1.15185,1.15133,1.15180,330,0


Plot that day

In [67]:
import plotly.graph_objects as go


fig = go.Figure(
    data=go.Candlestick(
        x=day.index,
        open=day["open"],
        high=day["high"],
        low=day["low"],
        close=day["close"],
    )
)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()
